## Script used to download my personal stats from my favorite mobile game Clash of Clans  
Author: Luis Mejia
<ol>
  <li>Imports and declarations</li>
  <li>Make request to Clash of Clans API</li>
  <li>Scrape JSON file and load data into their own DataFrame</li>
  <li>Loop through the DataFrames and insert them into my personal Google BigQuery project</li>
  <li>You can add a list of clash accounts and loop through them and get the stats for each, I only needed mine, hence this simple code.</li>
</ol>

In [6]:
import requests
import json
import pandas as pd
from datetime import date
from tabulate import tabulate
from google.cloud import bigquery
api_token = 'the key assigned to me by Clash of Clans'
player = '23L29CPQL8' #This is my Clash of Clans Townhall 15
today = date.today()
client = bigquery.Client.from_service_account_json(r'service_account.json')

In [7]:
#Make COC API request
url = "https://api.clashofclans.com/v1/players/%{player}".format(player=player)
r = requests.get(url, headers={"Accept":"application/json", "authorization":"Bearer "+api_token}) #, params = {"limit":20})
data = json.loads(r.text)

In [8]:
#Get data from JSON and put into it's own DataFrame, also add player Tag and Archive date to tie everything together
play_stats = pd.DataFrame.from_dict(data, orient='index')
play_stats = play_stats.transpose()
play_stats = play_stats[play_stats.columns[:19]]
play_stats[['archive_date','player_tag']] = pd.DataFrame([[today,player]],index=play_stats.index)
play_stats.Name = 'play_stats'

achievements = pd.DataFrame(data['achievements'])
achievements[['archive_date','player_tag']] = pd.DataFrame([[today,player]],index=achievements.index)
achievements.Name = 'achievements'

troops = pd.DataFrame(data['troops'])
troops[['archive_date','player_tag']] = pd.DataFrame([[today,player]],index=troops.index)
troops.Name = 'troops'

heroes = pd.DataFrame(data['heroes'])
heroes[['archive_date','player_tag']] = pd.DataFrame([[today,player]],index=heroes.index)
heroes.Name = 'heroes'

spells = pd.DataFrame(data['spells'])
spells[['archive_date','player_tag']] = pd.DataFrame([[today,player]],index=spells.index)
spells.Name = 'spells'

In [9]:
#Reusable code to import a dataframe into a BigQuery table
def import_to_gbq(df, destination_table):
    try:
        ## If you want the column and row count then
        table_id = destination_table
        job_config = bigquery.LoadJobConfig(
            schema=[
                #I can specify the column data type here, but for this scenario, BigQuery automatically detects the right data types, if it ain't broke, don't fix it!
            ],
            write_disposition="WRITE_TRUNCATE",
        )

        job = client.load_table_from_dataframe(
            df, table_id, job_config=job_config
        )

        job.result()  # Wait for the job to complete.
        table = client.get_table(table_id)
        print("Loaded {} rows and {} columns to {}".format(table.num_rows, len(table.schema), table_id))
    except Exception as e:
        print(e)

In [10]:
#loop through all frames and insert them in GBQ
all_frames = [play_stats,achievements,troops,heroes,spells]
for frame in all_frames:
    frame_name = frame.Name
    if frame_name == 'play_stats':
        import_to_gbq(frame, 'fleet-parser-330316.luis_stage.stg_coc_player_stats')
    elif frame_name == 'achievements':
        import_to_gbq(frame, 'fleet-parser-330316.luis_stage.stg_coc_player_achievements')
    elif frame_name == 'troops':
        print(tabulate(frame, headers='keys', tablefmt='simple_outline')) ##fancy_outline, simple_outline, simple_grid, rounded_grid, fancy_grid, html
        import_to_gbq(frame, 'fleet-parser-330316.luis_stage.stg_coc_player_troops')
    elif frame_name == 'heroes':
        print(tabulate(frame, headers='keys', tablefmt='simple_outline')) ##fancy_outline, simple_outline, simple_grid, rounded_grid, fancy_grid, html
        import_to_gbq(frame, 'fleet-parser-330316.luis_stage.stg_coc_player_heroes')
    elif frame_name == 'spells':
        print(tabulate(frame, headers='keys', tablefmt='simple_outline')) ##fancy_outline, simple_outline, simple_grid, rounded_grid, fancy_grid, html
        import_to_gbq(frame, 'fleet-parser-330316.luis_stage.stg_coc_player_spells')
    else:
        'nothing'                        
    

Loaded 1 rows and 21 columns to fleet-parser-330316.luis_stage.stg_coc_player_stats
Loaded 43 rows and 9 columns to fleet-parser-330316.luis_stage.stg_coc_player_achievements
┌────┬────────────────────┬─────────┬────────────┬─────────────┬────────────────┬──────────────┐
│    │ name               │   level │   maxLevel │ village     │ archive_date   │ player_tag   │
├────┼────────────────────┼─────────┼────────────┼─────────────┼────────────────┼──────────────┤
│  0 │ Barbarian          │      11 │         11 │ home        │ 2022-12-03     │ 23L29CPQL8   │
│  1 │ Archer             │      11 │         11 │ home        │ 2022-12-03     │ 23L29CPQL8   │
│  2 │ Goblin             │       8 │          8 │ home        │ 2022-12-03     │ 23L29CPQL8   │
│  3 │ Giant              │      11 │         11 │ home        │ 2022-12-03     │ 23L29CPQL8   │
│  4 │ Wall Breaker       │      11 │         11 │ home        │ 2022-12-03     │ 23L29CPQL8   │
│  5 │ Balloon            │      10 │         10 

# Thank you!